# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [43]:
# your code here
orders = pd.read_csv('Orders.csv')

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [17]:
orders['CustomerID'] =  orders['CustomerID'].astype('object')

# CustomerID are set as numerical. We need to convert them to object type.

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
count,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000
mean,278465.221859,560617.126645,2010.934259,7.612537,3.614555,12.728247,13.021823,3.116174,22.394749
std,152771.368303,13106.167695,0.247829,3.416527,1.928274,2.273535,180.420210,22.096788,309.055588
min,0.000000,536365.000000,2010.000000,1.000000,1.000000,6.000000,1.000000,0.000000,0.000000
25%,148333.750000,549234.000000,2011.000000,5.000000,2.000000,11.000000,2.000000,1.250000,4.680000
50%,284907.500000,561893.000000,2011.000000,8.000000,3.000000,13.000000,6.000000,1.950000,11.800000
75%,410079.250000,572090.000000,2011.000000,11.000000,5.000000,14.000000,12.000000,3.750000,19.800000
max,541908.000000,581587.000000,2011.000000,12.000000,7.000000,20.000000,80995.000000,8142.750000,168469.600000


In [45]:
customers = orders.groupby(['CustomerID']).agg({'amount_spent':sum})
spent_q95 = np.quantile(customers, q=0.95)
spent_q75 = np.quantile(customers, q=0.75)

customers = customers.rename(columns={'amount_spent':'total_amount_spent'})
def cust_status(x):
    if x < spent_q75: return "Hobo"
    elif x > spent_q95: return "VIP"
    else: return "Preferred"

customers['status'] = customers['total_amount_spent'].apply(cust_status)

customers.status.value_counts()

# we are getting a list of customers with aggregate spending
# then we calculating spending for quantile 95 and 75 
# after that we define a function to set a label to each customer

Hobo         3254
Preferred     868
VIP           217
Name: status, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [98]:
orders_status = pd.merge(left = orders,
                                  right = customers,
                                  how = 'left', 
                                  left_on = "CustomerID", 
                                  right_on= "CustomerID") #merging orders and status data

countries_vip = orders_status.groupby(['Country','status']).agg({'CustomerID':pd.Series.nunique}).reset_index() #grouping
vip = countries_vip[countries_vip['status']=='VIP'].sort_values('CustomerID',ascending=False) 
vip

,Country,status,CustomerID
74,United Kingdom,VIP,177
33,Germany,VIP,10
30,France,VIP,9
69,Switzerland,VIP,3
63,Spain,VIP,2
57,Portugal,VIP,2
2,Australia,VIP,2
23,EIRE,VIP,2
43,Japan,VIP,2
27,Finland,VIP,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [97]:
preferred = countries_vip[countries_vip['status']=='Preferred'].sort_values('CustomerID',ascending=False)

countries = pd.DataFrame(orders['Country'].unique()) # starting a table with full list of countries

countries = countries.rename(columns = {0:'Country'}) 

total_vip = pd.merge(left = countries,
                                  right = vip,
                                  how = 'left', 
                                  left_on = "Country", 
                                  right_on= "Country") #merging with VIP data

total_vip_pref = pd.merge(left = total_vip,
                                  right = preferred,
                                  how = 'left', 
                                  left_on = "Country", 
                                  right_on= "Country") #merging with Preferred data


total_vip_pref = total_vip_pref.drop(["status_x","status_y"], axis = 1) #dropping odd columns
total_vip_pref.columns = ['Country','VIP','Preferred'] # renaming columns
total_vip_pref.fillna(0, inplace = True) # filling voids

total_vip_pref['VIP+Preferred']= total_vip_pref["VIP"]+total_vip_pref["Preferred"] #vip+preferred column
total_vip_pref.sort_values('VIP+Preferred',ascending=False) #final result



,Country,VIP,Preferred,VIP+Preferred
0,United Kingdom,177.0,755.0,932.0
4,Germany,10.0,29.0,39.0
1,France,9.0,20.0,29.0
12,Belgium,1.0,11.0,12.0
7,Switzerland,3.0,6.0,9.0
8,Spain,2.0,7.0,9.0
5,Norway,1.0,6.0,7.0
10,Portugal,2.0,5.0,7.0
20,Finland,1.0,4.0,5.0
11,Italy,0.0,5.0,5.0
